# Initialize Packages

In [2]:
# Imports
import geopandas as gpd
import pandas as pd
import numpy as np
import numba
from numba import njit
import logging
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pvlib
from pathos.multiprocessing import ProcessingPool as Pool
import multiprocessing as mp  # Still import if needed for cpu_count()
from multiprocessing import Pool, cpu_count
from datetime import datetime
import tqdm
import fiona

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Functions and Class Definitions

### Shadow Optimization

In [3]:
@numba.njit
def calculate_shadow_impact_numba(height_diff_array, sun_altitude, distance_array):
    """Calculate the shadow impact for arrays of buildings."""
    impacts = np.zeros_like(height_diff_array)
    tan_sun_altitude = np.tan(np.radians(sun_altitude))
    for i in range(len(height_diff_array)):
        height_diff = height_diff_array[i]
        distance = distance_array[i]

        if sun_altitude <= 0:
            impacts[i] = 1.0
            continue
        if height_diff <= 0:
            impacts[i] = 0.0
            continue

        shadow_length = height_diff / tan_sun_altitude
        if distance <= shadow_length:
            impacts[i] = 1.0 - (distance / shadow_length)
        else:
            impacts[i] = 0.0

    return impacts

### Shadow Vectorization Definition

In [4]:
def calculate_shadow_impact_vectorized(building_row, buildings_gdf, solar_pos, spatial_index):
    """Calculate shadow impact for a building considering nearby buildings."""
    try:
        MAX_SHADOW_DISTANCE = 3 * building_row['heightroof']
        bounds = building_row.geometry.bounds

        # Expand bounds by MAX_SHADOW_DISTANCE
        bounds = (
            bounds[0] - MAX_SHADOW_DISTANCE,
            bounds[1] - MAX_SHADOW_DISTANCE,
            bounds[2] + MAX_SHADOW_DISTANCE,
            bounds[3] + MAX_SHADOW_DISTANCE
        )

        # Use spatial index to find nearby buildings
        possible_matches_idx = list(spatial_index.intersection(bounds))
        nearby_buildings = buildings_gdf.iloc[possible_matches_idx]
        nearby_buildings = nearby_buildings[nearby_buildings.index != building_row.name]

        if nearby_buildings.empty:
            return 1.0

        ref_centroid = building_row.geometry.centroid.coords[0]
        nearby_centroids = np.array([geom.centroid.coords[0] for geom in nearby_buildings.geometry])

        dx = nearby_centroids[:, 0] - ref_centroid[0]
        dy = nearby_centroids[:, 1] - ref_centroid[1]
        distances = np.sqrt(dx**2 + dy**2)

        height_diff = nearby_buildings['heightroof'].values - building_row['heightroof']
        sun_altitude = solar_pos['apparent_elevation']

        shadow_impacts = calculate_shadow_impact_numba(
            height_diff,
            sun_altitude,
            distances
        )

        total_shadow_impact = np.mean(shadow_impacts)
        shadow_factor = max(0.0, 1.0 - total_shadow_impact)
        return shadow_factor

    except Exception as e:
        logger.error(f"Error calculating shadow impact for building {building_row.name}: {str(e)}")
        return 1.0

### Worker Function

In [5]:
# Worker function at the module level
def worker_process_building(args):
    """Worker function to process a single building."""
    idx, building, analyzer = args
    area_m2 = building['Shape_Area'] * 0.092903  # Convert from square feet to square meters

    # Calculate the shadow factor
    shadow_factor = calculate_shadow_impact_vectorized(
        building, analyzer.buildings_gdf, analyzer.get_solar_position(), analyzer.spatial_index
    )

    solar_potential = (
        analyzer._annual_radiation *
        area_m2 *
        analyzer.PANEL_DENSITY *
        analyzer.PANEL_EFFICIENCY *
        analyzer.PERFORMANCE_RATIO *
        shadow_factor
    )

    return {
        'solar_potential': float(solar_potential),
        'effective_area': float(area_m2 * analyzer.PANEL_DENSITY),
        'peak_power': float(area_m2 * analyzer.PANEL_DENSITY * analyzer.PANEL_EFFICIENCY),
        'shadow_factor': float(shadow_factor),
        'annual_radiation': float(analyzer._annual_radiation),
        'performance_ratio': float(analyzer.PERFORMANCE_RATIO)
    }

### SolarAnalyzer Class Definition

In [6]:
# Class definition
class SolarAnalyzer:
    def __init__(self):
        logger.info("Initializing SolarAnalyzer...")
        self._initialize_constants()
        self._initialize_cache()
        self.buildings_gdf = None

    def _initialize_constants(self):
        self.NYC_LAT = 40.7128
        self.NYC_LON = -74.0060
        self.PANEL_EFFICIENCY = 0.20
        self.PERFORMANCE_RATIO = 0.75
        self.PANEL_DENSITY = 0.70
        self._solar_position = None
        self._spatial_index = None

    def _initialize_cache(self):
        self._monthly_radiation = self._initialize_radiation_data()
        self._annual_radiation = self._calculate_annual_radiation()

    def _initialize_radiation_data(self):
        """Initialize default radiation data (kWh/m²/day)."""
        return {
            '01': 2.45, '02': 3.42, '03': 4.53, '04': 5.64,
            '05': 6.48, '06': 6.89, '07': 6.75, '08': 5.98,
            '09': 4.92, '10': 3.67, '11': 2.56, '12': 2.12
        }

    def _calculate_annual_radiation(self):
        """Calculate annual radiation from monthly data (kWh/m²/year)."""
        monthly_sum = sum(self._monthly_radiation.values())
        return monthly_sum * 365 / 12

    def initialize_spatial_index(self, buildings_gdf):
        """Initialize spatial index for buildings."""
        self.spatial_index = buildings_gdf.sindex
        return buildings_gdf

    def get_solar_position(self):
        """Calculate solar position at a specific time."""
        if self._solar_position is None:
            times = pd.date_range('2020-06-21 12:00:00', periods=1, freq='H', tz='UTC')  # Summer solstice at noon
            location = pvlib.location.Location(latitude=self.NYC_LAT, longitude=self.NYC_LON)
            self._solar_position = location.get_solarposition(times).iloc[0]
        return self._solar_position

### Function for processing buildings in parallel

In [7]:
# Module-level function for processing buildings in parallel
def process_buildings_parallel(analyzer, buildings_chunk, buildings_gdf):
    """Process buildings using multiprocessing Pool without progress bar."""
    analyzer.buildings_gdf = buildings_gdf  # Store for access in worker functions
    analyzer.initialize_spatial_index(buildings_gdf)  # Ensure spatial index is initialized

    num_processes = max(1, cpu_count() - 1)
    logger.info(f"Using {num_processes} processes for parallel processing.")

    # Prepare the list of arguments for the worker function
    args_list = [(idx, row, analyzer) for idx, row in buildings_chunk.iterrows()]

    # Use the standard multiprocessing Pool without tqdm
    with Pool(processes=num_processes) as pool:
        # Map the worker function over the list of arguments
        results_list = pool.map(worker_process_building, args_list)

    # Create DataFrame from results
    results_df = pd.DataFrame(results_list, index=buildings_chunk.index)

    return results_df

# Function to analyze solar potential
def analyze_solar_potential(candidate_buildings, full_buildings):
    """
    Main function to analyze solar potential for candidate buildings.
    """
    try:
        # Initialize analyzer
        analyzer = SolarAnalyzer()
        logger.info("Initialized SolarAnalyzer")

        # Validate CRS
        if candidate_buildings.crs is None or full_buildings.crs is None:
            logger.warning("Input data missing CRS, assuming EPSG:4326")
            candidate_buildings = candidate_buildings.set_crs(epsg=4326)
            full_buildings = full_buildings.set_crs(epsg=4326)

        # Project to a suitable CRS for distance calculations (e.g., EPSG:2263)
        candidate_projected = candidate_buildings.to_crs(epsg=2263)
        buildings_projected = full_buildings.to_crs(epsg=2263)

        # Initialize spatial index
        analyzer.initialize_spatial_index(buildings_projected)

        # Process buildings using parallel processing
        results = process_buildings_parallel(
            analyzer,
            candidate_projected,
            buildings_projected
        )

        # Merge results back to original GeoDataFrame
        analyzed_buildings = candidate_projected.join(results)

        # Reproject back to original CRS
        return analyzed_buildings.to_crs(candidate_buildings.crs)

    except Exception as e:
        logger.error(f"Error in solar potential analysis: {str(e)}")
        return None

# Prepare and Process Candidate Buildings

In [8]:
def filter_pois(pois):
    target_classes = [
        'arts_centre', 'college', 'community_centre',
        'kindergarten', 'library', 'school', 'shelter'
    ]
    filtered_pois = pois[pois['fclass'].isin(target_classes)].copy()
    print(f"\nFiltered POIs by class:")
    print(filtered_pois['fclass'].value_counts())
    return filtered_pois

def merge_candidates(filtered_pois, pofw):
    # Prepare POFW by adding prefix to fclass
    pofw_prep = pofw.copy()
    pofw_prep['fclass'] = 'pofw_' + pofw_prep['fclass']

    # Select columns to keep
    cols_to_keep = ['fclass', 'name', 'geometry']

    # Merge datasets
    candidates = pd.concat([
        filtered_pois[cols_to_keep],
        pofw_prep[cols_to_keep]
    ], ignore_index=True)

    # Add ObjectID
    candidates['ObjectID'] = candidates.index + 1

    print(f"\nMerged candidates by class:")
    print(candidates['fclass'].value_counts())
    return candidates

Load Data

In [9]:
# Load data
# Set base input directory
INPUT_DIR = '/Users/oliveratwood/One Architecture Dropbox/Oliver Atwood/P2415_CSC Year Two/05 GIS/06 Scripts/ResilienceHub/Input'

# Derive paths for input files
gdb_path = f"{INPUT_DIR}/ResilienceHub.gdb"

def load_gdb_layers():
    # List all layers in the geodatabase
    layers = fiona.listlayers(gdb_path)
    print(f"Available layers in geodatabase: {layers}")

    # Load required layers
    buildings = gpd.read_file(gdb_path, layer='NYC_Buildings')
    pois = gpd.read_file(gdb_path, layer='NYC_POIS')
    pofw = gpd.read_file(gdb_path, layer='NYC_POFW')

    return buildings, pois, pofw

# Load the data
# buildings, pois, pofw, nsi = load_gdb_layers()
buildings, pois, pofw = load_gdb_layers()

print("\nDataset Summary:")
print(f"Number of buildings: {len(buildings):,}")
print(f"Number of Places of Interest: {len(pois):,}")
print(f"Number of Places of Worship: {len(pofw):,}")

# Display sample of each dataset
print("\nBuildings sample columns:")
print(buildings.columns.tolist())
print("\nPOIs sample columns:")
print(pois.columns.tolist())

# Process candidates
filtered_pois = filter_pois(pois)
candidate_points = merge_candidates(filtered_pois, pofw)

# Spatial join to get candidate buildings
candidate_buildings = gpd.sjoin(
    buildings,
    candidate_points,
    how='inner',
    predicate='contains'
)

print(f"\nFinal candidate buildings:")
print(f"Total buildings selected: {len(candidate_buildings):,}")
print("\nDistribution by facility type:")
print(candidate_buildings['fclass'].value_counts())

# Ensure required columns are present
required_columns = ['Shape_Area', 'heightroof', 'geometry']
missing_cols = [col for col in required_columns if col not in candidate_buildings.columns]
if missing_cols:
    raise ValueError(f"Missing required columns in candidate_buildings: {missing_cols}")

Available layers in geodatabase: ['NYC_POIS', 'NYC_NSI', 'NYC_Buildings', 'NYC_BBox', 'NYC_POFW', 'FDNY_Firehouses']

Dataset Summary:
Number of buildings: 1,084,413
Number of Places of Interest: 25,292
Number of Places of Worship: 1,106

Buildings sample columns:
['base_bbl', 'bin', 'cnstrct_yr', 'doitt_id', 'feat_code', 'geomsource', 'groundelev', 'heightroof', 'date_lstmo', 'time_lstmo', 'lststatype', 'mpluto_bbl', 'name', 'Shape_Length', 'Shape_Area', 'geometry']

POIs sample columns:
['osm_id', 'code', 'fclass', 'name', 'geometry']

Filtered POIs by class:
fclass
school              718
kindergarten        115
arts_centre          83
community_centre     68
library              56
college              22
shelter               7
Name: count, dtype: int64

Merged candidates by class:
fclass
pofw_christian                731
school                        718
kindergarten                  115
pofw_jewish                   112
arts_centre                    83
pofw_christian_catholic  

In [10]:
# Analyze solar potential with parallel processing
if __name__ == "__main__":

    # Sample candidate buildings for testing
    sample_size = 10
    candidate_buildings_sample = candidate_buildings.sample(sample_size, random_state=42)

    analyzed_buildings = analyze_solar_potential(candidate_buildings_sample, buildings)

    # Check results
    if analyzed_buildings is not None and not analyzed_buildings.empty:
        print("\nSolar analysis completed successfully.")
        print(f"Number of buildings analyzed: {len(analyzed_buildings)}")

        # Display summary statistics
        print("\nSummary of solar potential:")
        print(analyzed_buildings['solar_potential'].describe())

        # Display shadow factor statistics
        print("\nShadow Factor Statistics:")
        print(analyzed_buildings['shadow_factor'].describe())

    else:
        print("Solar analysis returned no results.")

INFO:__main__:Initializing SolarAnalyzer...
INFO:__main__:Initialized SolarAnalyzer
INFO:__main__:Using 9 processes for parallel processing.
Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'worker_process_building' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Users/oliveratwood/mamba

KeyboardInterrupt: 

In [ ]:
# Import custom module
from solar_mapping import create_detailed_solar_map

# Create and display the map
solar_map = create_detailed_solar_map(analyzed_buildings)

# Save the map
output_dir = Path('results')
output_dir.mkdir(exist_ok=True)
map_file = output_dir / 'solar_potential_map.html'
solar_map.save(str(map_file))

# Display the map in the notebook
solar_map

NameError: name 'analyzed_buildings' is not defined

In [ ]:
stop

In [ ]:
# Global variables for worker processes
global_buildings_gdf = None
global_spatial_index = None
global_solar_position = None

def init_worker(buildings_gdf, spatial_index, solar_position):
    """Initialize global variables in worker processes."""
    global global_buildings_gdf
    global global_spatial_index
    global global_solar_position
    global_buildings_gdf = buildings_gdf
    global_spatial_index = spatial_index
    global_solar_position = solar_position

def process_buildings_parallel(analyzer, buildings_chunk, buildings_gdf):
    """Process buildings using multiprocessing Pool without progress bar."""
    analyzer.buildings_gdf = buildings_gdf  # Store for access in worker functions
    analyzer.initialize_spatial_index(buildings_gdf)  # Ensure spatial index is initialized

    num_processes = max(1, cpu_count() - 1)
    logger.info(f"Using {num_processes} processes for parallel processing.")

    # Prepare the list of arguments for the worker function
    args_list = [(idx, row, analyzer) for idx, row in buildings_chunk.iterrows()]

    # Use the standard multiprocessing Pool without tqdm
    with Pool(processes=num_processes) as pool:
        # Map the worker function over the list of arguments
        results_list = pool.map(worker_process_building, args_list)

    # Create DataFrame from results
    results_df = pd.DataFrame(results_list, index=buildings_chunk.index)

    return results_df

def worker_process_building(args):
    """Worker function to process a single building using global variables."""
    idx, building = args
    area_m2 = building['Shape_Area'] * 0.092903  # Convert from square feet to square meters

    # Access global variables
    buildings_gdf = global_buildings_gdf
    spatial_index = global_spatial_index
    solar_pos = global_solar_position

    # Calculate the shadow factor
    shadow_factor = calculate_shadow_impact_vectorized(
        building, buildings_gdf, solar_pos, spatial_index
    )

    # Perform calculations
    solar_potential = (
        annual_radiation *
        area_m2 *
        PANEL_DENSITY *
        PANEL_EFFICIENCY *
        PERFORMANCE_RATIO *
        shadow_factor
    )

    return {
        'solar_potential': float(solar_potential),
        'effective_area': float(area_m2 * PANEL_DENSITY),
        'peak_power': float(area_m2 * PANEL_DENSITY * PANEL_EFFICIENCY),
        'shadow_factor': float(shadow_factor),
        'annual_radiation': float(annual_radiation),
        'performance_ratio': float(PERFORMANCE_RATIO)
    }

In [ ]:
class SolarAnalyzer:
    def __init__(self):
        logger.info("Initializing SolarAnalyzer...")
        self._initialize_constants()
        self._initialize_cache()
        self.buildings_gdf = None  # Add this to store the full buildings GeoDataFrame

    def _initialize_constants(self):
        self.NYC_LAT = 40.7128
        self.NYC_LON = -74.0060
        self.PANEL_EFFICIENCY = 0.20
        self.PERFORMANCE_RATIO = 0.75
        self.PANEL_DENSITY = 0.70
        self._solar_position = None
        self._spatial_index = None

    def _initialize_cache(self):
        self._monthly_radiation = self._initialize_radiation_data()
        self._annual_radiation = self._calculate_annual_radiation()

    def _initialize_radiation_data(self):
        """Initialize default radiation data (kWh/m²/day)."""
        return {
            '01': 2.45, '02': 3.42, '03': 4.53, '04': 5.64,
            '05': 6.48, '06': 6.89, '07': 6.75, '08': 5.98,
            '09': 4.92, '10': 3.67, '11': 2.56, '12': 2.12
        }

    def _calculate_annual_radiation(self):
        """Calculate annual radiation from monthly data (kWh/m²/year)."""
        monthly_sum = sum(self._monthly_radiation.values())
        return monthly_sum * 365 / 12

    def initialize_spatial_index(self, buildings_gdf):
        """Initialize spatial index for buildings."""
        self.spatial_index = buildings_gdf.sindex
        return buildings_gdf

    def process_buildings_parallel(analyzer, buildings_chunk, buildings_gdf):
        """Process buildings using multiprocessing Pool with minimal progress output."""
        analyzer.buildings_gdf = buildings_gdf  # Store for access in worker functions
        analyzer.initialize_spatial_index(buildings_gdf)  # Ensure spatial index is initialized

        num_processes = max(1, cpu_count() - 1)
        logger.info(f"Using {num_processes} processes for parallel processing.")

        # Prepare the list of arguments for the worker function
        args_list = [(idx, row, analyzer) for idx, row in buildings_chunk.iterrows()]

        total_tasks = len(args_list)
        chunk_size = max(1, total_tasks // 10)  # Adjust chunk size as needed

        # Use the standard multiprocessing Pool with a simple progress indicator
        with Pool(processes=num_processes) as pool:
            results_list = []
            for i, result in enumerate(pool.imap(worker_process_building, args_list, chunksize=chunk_size), 1):
                results_list.append(result)
                if i % chunk_size == 0 or i == total_tasks:
                    logger.info(f"Processed {i}/{total_tasks} buildings...")

        # Create DataFrame from results
        results_df = pd.DataFrame(results_list, index=buildings_chunk.index)

        return results_df

    def _process_single_building_wrapper(self, args):
        """Wrapper function to process a single building."""
        idx, building = args
        result = self._process_single_building(building)
        return result  # No need to include 'index'

    def get_solar_position(self):
        """Calculate solar position at a specific time."""
        if self._solar_position is None:
            times = pd.date_range('2020-06-21 12:00:00', periods=1, freq='H', tz='UTC')  # Summer solstice at noon
            location = pvlib.location.Location(latitude=self.NYC_LAT, longitude=self.NYC_LON)
            self._solar_position = location.get_solarposition(times).iloc[0]
        return self._solar_position

    def _process_single_building(self, building):
        """Process a single building."""
        area_m2 = building['Shape_Area'] * 0.092903  # Convert from square feet to square meters

        # Calculate the shadow factor
        shadow_factor = self.calculate_shadow_impact_vectorized(
            building, self.buildings_gdf, self.get_solar_position()
        )

        solar_potential = (
            self._annual_radiation *
            area_m2 *
            self.PANEL_DENSITY *
            self.PANEL_EFFICIENCY *
            self.PERFORMANCE_RATIO *
            shadow_factor
        )

        return {
            'solar_potential': float(solar_potential),
            'effective_area': float(area_m2 * self.PANEL_DENSITY),
            'peak_power': float(area_m2 * self.PANEL_DENSITY * self.PANEL_EFFICIENCY),
            'shadow_factor': float(shadow_factor),
            'annual_radiation': float(self._annual_radiation),
            'performance_ratio': float(self.PERFORMANCE_RATIO)
        }

Define the analyze_solar_potential Function

In [ ]:
def analyze_solar_potential(candidate_buildings, full_buildings):
    """
    Main function to analyze solar potential for candidate buildings.
    """
    try:
        # Initialize analyzer
        analyzer = SolarAnalyzer()
        logger.info("Initialized SolarAnalyzer")

        # Validate CRS
        if candidate_buildings.crs is None or full_buildings.crs is None:
            logger.warning("Input data missing CRS, assuming EPSG:4326")
            candidate_buildings = candidate_buildings.set_crs(epsg=4326)
            full_buildings = full_buildings.set_crs(epsg=4326)

        # Project to a suitable CRS for distance calculations (e.g., EPSG:2263)
        candidate_projected = candidate_buildings.to_crs(epsg=2263)
        buildings_projected = full_buildings.to_crs(epsg=2263)

        # Initialize spatial index
        analyzer.initialize_spatial_index(buildings_projected)

        # Process buildings using parallel processing
        results = process_buildings_parallel(
            analyzer,
            candidate_projected,
            buildings_projected
        )

        # Merge results back to original GeoDataFrame
        analyzed_buildings = candidate_projected.join(results)

        # Reproject back to original CRS
        return analyzed_buildings.to_crs(candidate_buildings.crs)

    except Exception as e:
        logger.error(f"Error in solar potential analysis: {str(e)}")
        return None

Prepare and Process Candidate Buildings

In [ ]:
def filter_pois(pois):
    target_classes = [
        'arts_centre', 'college', 'community_centre',
        'kindergarten', 'library', 'school', 'shelter'
    ]
    filtered_pois = pois[pois['fclass'].isin(target_classes)].copy()
    print(f"\nFiltered POIs by class:")
    print(filtered_pois['fclass'].value_counts())
    return filtered_pois

def merge_candidates(filtered_pois, pofw):
    # Prepare POFW by adding prefix to fclass
    pofw_prep = pofw.copy()
    pofw_prep['fclass'] = 'pofw_' + pofw_prep['fclass']

    # Select columns to keep
    cols_to_keep = ['fclass', 'name', 'geometry']

    # Merge datasets
    candidates = pd.concat([
        filtered_pois[cols_to_keep],
        pofw_prep[cols_to_keep]
    ], ignore_index=True)

    # Add ObjectID
    candidates['ObjectID'] = candidates.index + 1

    print(f"\nMerged candidates by class:")
    print(candidates['fclass'].value_counts())
    return candidates

Load Data

In [ ]:
# Load data
# Set base input directory
INPUT_DIR = '/Users/oliveratwood/One Architecture Dropbox/Oliver Atwood/P2415_CSC Year Two/05 GIS/06 Scripts/ResilienceHub/Input'

# Derive paths for input files
gdb_path = f"{INPUT_DIR}/ResilienceHub.gdb"

def load_gdb_layers():
    # List all layers in the geodatabase
    layers = fiona.listlayers(gdb_path)
    print(f"Available layers in geodatabase: {layers}")

    # Load required layers
    buildings = gpd.read_file(gdb_path, layer='NYC_Buildings')
    pois = gpd.read_file(gdb_path, layer='NYC_POIS')
    pofw = gpd.read_file(gdb_path, layer='NYC_POFW')

    return buildings, pois, pofw

# Load the data
# buildings, pois, pofw, nsi = load_gdb_layers()
buildings, pois, pofw = load_gdb_layers()

print("\nDataset Summary:")
print(f"Number of buildings: {len(buildings):,}")
print(f"Number of Places of Interest: {len(pois):,}")
print(f"Number of Places of Worship: {len(pofw):,}")

# Display sample of each dataset
print("\nBuildings sample columns:")
print(buildings.columns.tolist())
print("\nPOIs sample columns:")
print(pois.columns.tolist())

# Process candidates
filtered_pois = filter_pois(pois)
candidate_points = merge_candidates(filtered_pois, pofw)

# Spatial join to get candidate buildings
candidate_buildings = gpd.sjoin(
    buildings,
    candidate_points,
    how='inner',
    predicate='contains'
)

print(f"\nFinal candidate buildings:")
print(f"Total buildings selected: {len(candidate_buildings):,}")
print("\nDistribution by facility type:")
print(candidate_buildings['fclass'].value_counts())

# Ensure required columns are present
required_columns = ['Shape_Area', 'heightroof', 'geometry']
missing_cols = [col for col in required_columns if col not in candidate_buildings.columns]
if missing_cols:
    raise ValueError(f"Missing required columns in candidate_buildings: {missing_cols}")

Available layers in geodatabase: ['NYC_POIS', 'NYC_NSI', 'NYC_Buildings', 'NYC_BBox', 'NYC_POFW', 'FDNY_Firehouses']

Dataset Summary:
Number of buildings: 1,084,413
Number of Places of Interest: 25,292
Number of Places of Worship: 1,106

Buildings sample columns:
['base_bbl', 'bin', 'cnstrct_yr', 'doitt_id', 'feat_code', 'geomsource', 'groundelev', 'heightroof', 'date_lstmo', 'time_lstmo', 'lststatype', 'mpluto_bbl', 'name', 'Shape_Length', 'Shape_Area', 'geometry']

POIs sample columns:
['osm_id', 'code', 'fclass', 'name', 'geometry']

Filtered POIs by class:
fclass
school              718
kindergarten        115
arts_centre          83
community_centre     68
library              56
college              22
shelter               7
Name: count, dtype: int64

Merged candidates by class:
fclass
pofw_christian                731
school                        718
kindergarten                  115
pofw_jewish                   112
arts_centre                    83
pofw_christian_catholic  

# Analyze solar potential with parallel processing

In [ ]:
# Sample candidate buildings for testing
sample_size = 10
candidate_buildings_sample = candidate_buildings.sample(sample_size, random_state=42)

# Analyze solar potential with parallel processing
if __name__ == "__main__":

    analyzed_buildings = analyze_solar_potential(candidate_buildings_sample, buildings)

# Check results
if analyzed_buildings is not None and not analyzed_buildings.empty:
    print("\nSolar analysis completed successfully.")
    print(f"Number of buildings analyzed: {len(analyzed_buildings)}")

    # Display summary statistics
    print("\nSummary of solar potential:")
    print(analyzed_buildings['solar_potential'].describe())

    # Display shadow factor statistics
    print("\nShadow Factor Statistics:")
    print(analyzed_buildings['shadow_factor'].describe())

else:
    print("Solar analysis returned no results.")

INFO:__main__:Initializing SolarAnalyzer...
INFO:__main__:Initialized SolarAnalyzer
INFO:__main__:Using 9 processes for parallel processing.
Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/multiprocessing/queues.py", line 367, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'worker_process_building' on <module '__main__' (built-in)>
Process SpawnPoolWorker-2:
Traceback (most recent call last):
  File "/Users/oliveratwood/mamba

KeyboardInterrupt: 

Save and Visualize Results

In [ ]:
# Save results to a GeoJSON file
output_file = 'analyzed_candidate_buildings.geojson'
analyzed_buildings.to_file(output_file, driver='GeoJSON')
print(f"\nAnalyzed buildings saved to '{output_file}'")

INFO:pyogrio._io:Created 10 records



Analyzed buildings saved to 'analyzed_candidate_buildings.geojson'


In [ ]:
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()

# Your code to analyze solar potential
analyzed_buildings = analyze_solar_potential(candidate_buildings_sample, buildings)

profiler.disable()
stats = pstats.Stats(profiler).sort_stats('cumtime')
stats.print_stats(20)  # Print top 20 functions

INFO:__main__:Initializing SolarAnalyzer...
INFO:__main__:Initialized SolarAnalyzer
INFO:__main__:Using 9 processes for parallel processing.
Processing Buildings: 100%|██████████| 10/10 [06:23<00:00, 38.36s/it]


         48781 function calls (48622 primitive calls) in 391.880 seconds

   Ordered by: cumulative time
   List reduced from 742 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  391.880  195.940 /Users/oliveratwood/mambaforge/envs/resilience_hubs/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000  391.880  195.940 {built-in method builtins.exec}
        1    0.634    0.634  391.880  391.880 /var/folders/2z/nndw55750ksd3hg456d452m80000gn/T/ipykernel_65328/690209817.py:1(<module>)
        1    0.004    0.004  391.246  391.246 /var/folders/2z/nndw55750ksd3hg456d452m80000gn/T/ipykernel_65328/1960843054.py:1(analyze_solar_potential)
        1    0.000    0.000  383.949  383.949 /var/folders/2z/nndw55750ksd3hg456d452m80000gn/T/ipykernel_65328/2790845119.py:39(process_buildings_parallel)
       27    0.000    0.000  383.805   14.215 /Users/oliveratwood/mamb

In [ ]:
# Import custom module
from solar_mapping import create_detailed_solar_map

# Create and display the map
solar_map = create_detailed_solar_map(analyzed_buildings)

# Save the map
# output_dir = Path('results')
# output_dir.mkdir(exist_ok=True)
# map_file = output_dir / 'solar_potential_map.html'
# solar_map.save(str(map_file))

# Display the map in the notebook
solar_map

Creating solar potential map...


In [ ]:
# # Set paths and load data
# import geopandas as gpd
# import fiona
# import gc

# # Set base input directory
# INPUT_DIR = '/Users/oliveratwood/One Architecture Dropbox/Oliver Atwood/P2415_CSC Year Two/05 GIS/06 Scripts/ResilienceHub/Input'

# # Derive paths for input files
# gdb_path = f"{INPUT_DIR}/ResilienceHub.gdb"

# # Clear the Python kernel's memory cache
# gc.collect()

# def load_gdb_layers():
#     try:
#         # List all layers in the geodatabase
#         layers = fiona.listlayers(gdb_path)
#         print(f"Available layers in geodatabase: {layers}")

#         # Load required layers
#         buildings = gpd.read_file(gdb_path, layer='NYC_Buildings')
#         pois = gpd.read_file(gdb_path, layer='NYC_POIS')
#         pofw = gpd.read_file(gdb_path, layer='NYC_POFW')
#         firehouses = gpd.read_file(gdb_path, layer='FDNY_Firehouses', driver='OpenFileGDB')

#         # Verify the data load
#         print("\nVerifying data load:")
#         for name, data in [('buildings', buildings), ('pois', pois), 
#                           ('pofw', pofw), ('firehouses', firehouses)]:
#             print(f"\n{name.upper()} columns:")
#             print(data.columns.tolist())
#             print(f"Number of records: {len(data)}")

#         return buildings, pois, pofw, firehouses

#     except Exception as e:
#         print(f"Error loading data: {str(e)}")
#         raise

# # Load the data
# buildings, pois, pofw, firehouses = load_gdb_layers()

# # Print summary statistics
# print("\nDataset Summary:")
# print(f"Number of buildings: {len(buildings):,}")
# print(f"Number of Places of Interest: {len(pois):,}")
# print(f"Number of Places of Worship: {len(pofw):,}")
# print(f"Number of Firehouses: {len(firehouses):,}")


In [ ]:
# # Diagnostic code to examine firehouses dataset
# print("\nFirehouses Dataset Information:")
# print("Columns:")
# print(firehouses.columns.tolist())
# print("\nShape:", firehouses.shape)
# print("\nData Types:")
# print(firehouses.dtypes)

# # Try to display all columns, not just geometry
# print("\nFirst few rows with all columns:")
# print(firehouses.head().to_string())


# filter and prepare, including firehouses (and parking lots?)

In [ ]:
# # Filter and prepare candidate buildings
# def filter_pois(pois):
#     target_classes = [
#         'arts_centre', 'college', 'community_centre', 
#         'kindergarten', 'library', 'school', 'shelter'
#     ]
#     filtered_pois = pois[pois['fclass'].isin(target_classes)].copy()
#     print(f"\nFiltered POIs by class:")
#     print(filtered_pois['fclass'].value_counts())
#     return filtered_pois

# def prepare_firehouses(firehouses):
#     # Create a copy of the firehouses dataset
#     firehouses_prep = firehouses.copy()

#     # Add required columns with appropriate values
#     firehouses_prep['fclass'] = 'Firehouse'
#     firehouses_prep['name'] = firehouses_prep['FacilityName']

#     # Select only needed columns
#     cols_to_keep = ['fclass', 'name', 'geometry']
#     firehouses_prep = firehouses_prep[cols_to_keep]

#     print(f"\nPrepared firehouses:")
#     print(f"Total firehouses: {len(firehouses_prep)}")
#     return firehouses_prep

# def merge_candidates(filtered_pois, pofw, firehouses):
#     # Prepare POFW by adding prefix to fclass
#     pofw_prep = pofw.copy()
#     pofw_prep['fclass'] = 'pofw_' + pofw_prep['fclass']

#     # Prepare firehouses
#     firehouses_prep = prepare_firehouses(firehouses)

#     # Select columns to keep
#     cols_to_keep = ['fclass', 'name', 'geometry']

#     # Merge all three datasets
#     candidates = pd.concat([
#         filtered_pois[cols_to_keep],
#         pofw_prep[cols_to_keep],
#         firehouses_prep[cols_to_keep]
#     ], ignore_index=True)

#     # Add ObjectID
#     candidates['ObjectID'] = candidates.index + 1

#     print(f"\nMerged candidates by class:")
#     print(candidates['fclass'].value_counts())
#     return candidates

# # Process candidates
# filtered_pois = filter_pois(pois)
# candidate_points = merge_candidates(filtered_pois, pofw, firehouses)

# # Select building footprints
# candidate_buildings = gpd.sjoin(
#     buildings,
#     candidate_points,
#     how='inner',
#     predicate='contains'
# )

# print(f"\nFinal candidate buildings:")
# print(f"Total buildings selected: {len(candidate_buildings):,}")
# print("\nDistribution by facility type:")
# print(candidate_buildings['fclass'].value_counts())
